In [1]:
%load_ext blackcellmagic

# This notebook will:

* [Import Datalogue,](#Import) [connect and log in to Datalogue instance](#Connect)
* [Connect to a datastore with training data](#Connect_Store)
* [Create a relevant ontology](#Ontology)
* [Attach training data to the ontology](#Training_Data)
* [Train a model](#Train)
* [Attach data to be used in pipelining](#Data2)
* [Use that model in pipelines](#Pipelines)

# Import the SDK
<a id='Import'></a>

In [94]:
# Import Datalogue libraries 
from datalogue import *
from datalogue.version import __version__
from datalogue.models.ontology import *
from datalogue.models.datastore_collection import *
from datalogue.models.datastore import *
from datalogue.models.datastore import GCSDatastoreDef 
from datalogue.models.credentials import *
from datalogue.models.stream import *
from datalogue.models.transformations import *
from datalogue.models.transformations.structure import *
from datalogue.dtl import Dtl, DtlCredentials
from datalogue.models.training import DataRef

# Import Datalogue Bag of Tricks
from DTLBagOTricks import DTL as DTLHelper


# Import other useful libraries
from datetime import datetime, timedelta
from os import environ
import pandas
from IPython.display import Image

# Checks the version of the SDK is correct
# The expected version is 0.28.3
# If the SDK is not installed, run `! pip install datalogue` and restart the Jupyter Notebook kernel
# If the wrong versions is installed, run `! pip install datalogue --upgrade` and restart the Jupyter Notebook kernel
__version__

'0.28.3'

<a id='Connect'></a>
# Connect to Datalogue Install


Run these before launching jupyter:

``` bash
export DTL_EMAIL="your email"
export DTL_PASSWORD="your password"
```

This allows you to store your username and password outside of your notebook.

You may need to VPN into the machine that DTL is running on.


In [95]:
# Set host, username and password variables

datalogue_host = "http://prod.domain"  # for connecting to internal (note)

# datalogue_host = "https://internal.dtl.systems"  # for connecting to internal (note)
# datalogue_host = "http://10.2.161.119:3000"  # for connecting to Eric's DGX
email = environ.get("DTL_EMAIL")
password = environ.get("DTL_PASSWORD")

# Log in to Datalogue
BOT = DTLHelper(datalogue_host, email, password)
dtl = BOT.dtl

# Expected output Datalogue v0.28.3
# "Logged in '[host location]' with '[username]' account)"

Datalogue v0.28.3
Logged in 'http://prod.domain/api' with 'johanan@datalogue.io' account)


In [96]:
# Describe current server state (data stores and collections, active data streams)
BOT.server_summary()


Datalogue Server Summary :: Stores: 14, Collections: 4, Streams: 3



## Optional: clear old datastores and collections

<span style="color:red"> Warning! this is a non-reversible step.</span>

In [77]:
# Warning! this will clean all your datastores and data collections and credentials

# # Clear Datastores and Datastore Collections
# for store in dtl.datastore.list():
#     dtl.datastore.delete(store.id)
# for store in dtl.datastore_collection.list():
#     dtl.datastore_collection.delete(store.id)

# # Clear credentials
# for credential in dtl.credentials.admin.list():
#     dtl.credentials.admin.delete(credential.id)
    
# # Clear data pipelines
# for StreamCollection in dtl.stream_collection.list():
#     dtl.stream_collection.delete(StreamCollection.id)

## Clear ontologies
# for Ontology in dtl.ontology.list():
#     dtl.ontology.delete(ontology.id)

BOT.server_summary()

# After running the above, the Stores and Collections variables should both be 0

no pipelines available

Datalogue Server Summary :: Stores: 20, Collections: 3, Streams: 0



<a id='Connect_Store'></a>
# Attach relevant datastore(s) for use as training data
Here, we will:

1. add credentials to allow access to the data
2. connect to data stores
3. Collect the training data into a data collection

Note, this assumes that you are adding a CSV file from GCS as a data store, if you have other data store types you would like to add, see the [SDK Documentation](https://datalogue.readme.io/reference#datastore-1).


## Add credentials to Datastore

Run these before launching jupyter:

``` bash
export GCS_EMAIL="your email"
export GCS_PASSWORD="your key"
```

This allows you to store your username and password outside of your notebook.

Note: these aren't required if your datastore is a HTTP or Server Local File Store.

In [11]:
# Create local credential definition

gcs_credentials_def = GCS(
    client_email=environ.get("GCS_EMAIL"),
    private_key=environ.get("GCS_PASSWORD").replace("\\n", "\n"),
)

# Create credentials in datalogue platform

my_gcs_credentials = dtl.credentials.admin.create(
    credentials_definition=gcs_credentials_def, name="Telemetry GCS Credentials"
)

# Share credentials with all users of Datalogue deployment
# This is a necessary step even if you are the only user of the credentials

groups = dtl.group.get_list()
for group in groups:
    dtl.credentials.admin.share(my_gcs_credentials.id, group.id)


In [12]:
# Return table of credentials to verify above
BOT.get_credentials()

,credential_id,credential_name,credential_type
0,6487b551-c3b1-4e18-8811-eb3398e13797,Telemetry GCS Credentials,GCS
1,a33e2a29-dd74-473e-aef9-22a262e3a2ab,Telemetry GCS Credentials,GCS


## Attach a datastore for training data

In [13]:
# Attach a datastore

gcsdef_1 = GCSDatastoreDef(
    bucket = "dtl-handset-telemetry",
    file_name = "5K_us_telemetry.csv",
    file_format = FileFormat.Csv,
)

US_handset_data = dtl.datastore.create(
    Datastore(
        name = "US Telemetry 5k", 
        definition = gcsdef_1, 
        credential_id = my_gcs_credentials.id
    )
)


## Collecting training data into Datastore Collection

This collects the training data into a datastore collection for ease of reference.

In [14]:
# Create Datastore Collection objects to send to Datalogue, and send

dtl.datastore_collection.create(
    DatastoreCollection(
        name="Handset telemetry training data",
        description="Handset training data for use in model creation",
        storeIds=[US_handset_data.id],
    )
)

BOT.server_summary()


Datalogue Server Summary :: Stores: 1, Collections: 1, Streams: 1



![Datastore collection view](images/Datastore_collection.png)

Preview of the Datastore collection above in the Datalogue interface.

## Previewing datastore

In [15]:
# Preview datastore by loading 5 rows into a dataframe

Preview_US_Handset_table = dtl.datastore.load_arrow_table(
  US_handset_data.id, limit = 5
).to_pandas()

Preview_US_Handset_table.head()

# # print column names 
# for col in Preview_US_Handset_table.columns: 
#     print(col) 

,main_ConnectionType,main_ConnectionTechnology,main_ServiceProvider,main_ConnectionStart,main_ConnectionEnd,main_CID,main_LAC,main_BSSID,main_MCC,main_MNC,...,main_QOS_TA,main_QOS_SystemUptime,main_QOS_DeviceUsedStorage,main_QOS_DeviceUsedMemory,main_Country,main_Region,main_City,main_Geohash,conn_Generation_Category,sp_ServiceProviderBrandName
0,MOBILE,EVDOA,Boost Mobile,2017-06-06 18:22:07 UTC,2017-06-06 18:22:12 UTC,28339,248,None,None,4183,...,None,49674424,5121220608,743587840,United States,California,Stockton,9qc8p70trt,3G,None
1,MOBILE,LTE,Verizon Wireless,2017-06-16 22:41:48 UTC,2017-06-16 23:20:53 UTC,850,4,None,None,32,...,None,92000240,7030661120,1640054784,United States,Pennsylvania,null,dpqu2j7wrq,4G,None
2,MOBILE,EVDOA,Verizon Wireless,2017-05-12 19:34:02 UTC,2017-05-12 19:34:32 UTC,5937,1,None,None,1015,...,None,100459477,1835679744,737628160,United States,Georgia,Savannah,djwnzzpu8c,3G,None
3,MOBILE,EVDOA,,2017-06-29 19:22:10 UTC,2017-06-29 19:22:16 UTC,4964,4,None,None,20,...,None,189699713,5776637952,812548096,United States,Illinois,Rolling Meadows,dp3rsb194f,3G,None
4,MOBILE,LTE,Verizon Wireless,2017-05-18 06:10:30 UTC,2017-05-18 06:10:36 UTC,4880,5,None,None,32,...,None,374417638,3887247360,1100062720,United States,Pennsylvania,null,dpp51gwr26,4G,None


## Adding bulk web data for other classes

In [16]:
# Create a dictionary for each HTTP source to be added

data_HTTP_CSV = [
    {
        "store_name": "Canadian Names",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500canadians.csv",
    },
    {
        "store_name": "British Names",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500brits.csv",
    },
    {
        "store_name": "Australian Names",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500australians.csv",
    },
    {
        "store_name": "US Names",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500americans.csv",
    },
    {
        "store_name": "Users",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/pii/users.csv",
    },
    {
        "store_name": "Countries",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/countries.csv",
    },
    {
        "store_name": "Some Cities",
        "URL": "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/cities.csv",
    },
]

# This takes the above list of dictionaries and turns them into data stores

for data_store in data_HTTP_CSV:
    data_store["datastore_object"] = dtl.datastore.create(
        Datastore(
            data_store["store_name"],
            HttpDatastoreDef(data_store["URL"], FileFormat.Csv),
        )
    )

BOT.server_summary()



Datalogue Server Summary :: Stores: 8, Collections: 1, Streams: 1



In [17]:
# Create Dataset objects to send to Datalogue, and send

dtl.datastore_collection.create(
    DatastoreCollection(
        name="Supplemental training data (mainly customer data)",
        description="Additional data to enrich source",
        storeIds=[Datastore["datastore_object"].id for Datastore in data_HTTP_CSV],
    )
)

BOT.server_summary()



Datalogue Server Summary :: Stores: 8, Collections: 2, Streams: 1



In [18]:
# Checking that the collection was created

for dataset in dtl.datastore_collection.list():
    print('\033[1m' "Collection Name: ", dataset.name) 
    print('\033[0m' "             ID: ", dataset.id)
    print("       Contains: ", )
    for datastore in dataset.storeIds:
        print("                  ➜ " + datastore["name"])
        print("                    ↪ID: ", datastore["id"])
    print()


Collection Name:  Handset telemetry training data
             ID:  f7f9b5e4-b06f-4133-9d83-b943d18a953b
       Contains: 
                  ➜ US Telemetry 5k
                    ↪ID:  9e5aa44e-b321-44d6-b863-1577bcad7572

Collection Name:  Supplemental training data (mainly customer data)
             ID:  21e41c0e-304a-479c-8cb9-7005d25b946c
       Contains: 
                  ➜ US Names
                    ↪ID:  0000effb-eeda-43fd-8545-994ae1686335
                  ➜ Australian Names
                    ↪ID:  30a49e46-765c-4e30-9917-6476948e649f
                  ➜ Canadian Names
                    ↪ID:  3511a4b6-a498-4228-a765-878f2b710c80
                  ➜ Countries
                    ↪ID:  5212d5f9-91b7-4e1e-aa10-09a2e49a3bba
                  ➜ British Names
                    ↪ID:  b4770dfd-1078-4eaa-a8a8-09ec1faf1dd0
                  ➜ Users
                    ↪ID:  c7d8384f-2ef9-40d3-8850-c7d983cf7362
                  ➜ Some Cities
                    ↪ID:  fe346009-

![Datastore collection view](images/Datastore_collection2.png)

Preview of the Datastore collection above in the Datalogue interface.

<a id='Ontology'></a>
# Create ontology

In [19]:
wireless_ontology = Ontology(
    "Declassified Wireless Carrier Data",
    "This is for the purpose of cleaning and delivering safe data as a product.",
    [
        OntologyNode(
            "Sensitive Data",
            "This is data NOT to be distributed to 3rd parties.",
            [
                OntologyNode(
                    "Subscriber",
                    None,
                    [
                        OntologyNode("Full Name"),
                        OntologyNode("First Name"),
                        OntologyNode("Family Name"),
                        OntologyNode("Subscriber Company"),
                        OntologyNode("Account Identifier"),
                        OntologyNode(
                            "Address",
                            None,
                            [
                                OntologyNode("Subscriber Street"),
                                OntologyNode("Subscriber City"),
                                OntologyNode("Subscriber Zip"),
                            ],
                        ),
                        OntologyNode("Subscriber Phone Number"),
                        OntologyNode("Subscriber Email"),
                    ],
                ),
                OntologyNode(
                    "Sensitive Device Data",
                    None,
                    [
                        OntologyNode(
                            "Sensitive Device Identifier",
                            None,
                            [OntologyNode("Mac Address"), 
                             OntologyNode("IMEI")],
                        ),
                        OntologyNode(
                            "Sensitive Device Telemetry",
                            None,
                            [
                                OntologyNode(
                                    "Sensitive Geospatial",
                                    None,
                                    [
                                        OntologyNode("Latitude"),
                                        OntologyNode("Longitude"),
                                        OntologyNode("Altitude"),
                                        OntologyNode("Geo Hash"),
                                    ],
                                ),
                                OntologyNode("IP Address"),
                                OntologyNode("Mobile Network Code", "MNC"),
                            ],
                        ),
                    ],
                ),
                OntologyNode(
                    "Commercially Sensitive Information",
                    "Information that providers or manufacturers may not want released",
                    [
                        OntologyNode("Network Provider"),
                        OntologyNode("Manufacturer"),
                        OntologyNode("Device ID"),
                        OntologyNode("OS"),
                        OntologyNode("Device Model"),
                    ],
                ),
            ],
        ),
        OntologyNode(
            "Data to Distribute",
            "Wireless carrier data that is not sensitive and can be used for public consumption",
            [
                OntologyNode(
                    "Device Data",
                    None,
                    [
                        OntologyNode(
                            "Screen Resolution",
                            None,
                            [
                                OntologyNode("Screen Resolution Width"),
                                OntologyNode("Screen Resolution Height"),
                            ],
                        ),
                        OntologyNode("Device Memory"),
                        OntologyNode("Device Storage"),
                        OntologyNode("Device Language"),
                    ],
                ),
                OntologyNode(
                    "Device Telemetry",
                    None,
                    [
                        OntologyNode(
                            "Device Hardware Performance",
                            None,
                            [
                                OntologyNode("Device System Uptime"),
                                OntologyNode("Device Used Storage"),
                                OntologyNode("Device Unused Storage"),
                                OntologyNode("Device Used Memory"),
                                OntologyNode("Device Unused Memory"),
                                OntologyNode("Device CPU Usage"),
                                OntologyNode("Device Battery Usage"),
                                OntologyNode("Device Battery State"),
                            ],
                        ),
                        OntologyNode(
                            "Device Geospatial Data",
                            None,
                            [
                                OntologyNode(
                                    "Device Horizontal Accuracy",
                                    "Device Horizontal GPS Accuracy",
                                ),
                                OntologyNode(
                                    "Device Vertical Accuracy",
                                    "Device Vertical GPS Accuracy",
                                ),
                                OntologyNode("Device Velocity Speed"),
                                OntologyNode("Device Velocity Bearing"),
                                OntologyNode("Session State or Territory"),
                                OntologyNode("Session Country"),
                            ],
                        ),
                    ],
                ),
                OntologyNode(
                    "Network Telemetry",
                    None,
                    [
                        OntologyNode(
                            "Session Information",
                            None,
                            [
                                OntologyNode("Session Connection Type"),
                                OntologyNode("Session Connection Technology"),
                                OntologyNode(
                                    "Session Time",
                                    None,
                                    [
                                        OntologyNode("Session Start"),
                                        OntologyNode("Session End"),
                                        OntologyNode("Session Timezone"),
                                        OntologyNode("QoS Timestamp"),
                                    ],
                                ),
                                OntologyNode(
                                    "Session Delta Transmitted Bytes",
                                    "Delta from last session",
                                ),
                                OntologyNode(
                                    "Session Delta Received Bytes",
                                    "Delta from last session",
                                ),
                            ],
                        ),
                        OntologyNode(
                            "Mobile Network",
                            None,
                            [
                                OntologyNode("Mobile Connection Generation"),
                                OntologyNode("Mobile Channel"),
                                OntologyNode("Mobile Country Code", "MCC"),
                                OntologyNode("Base Station Identity Code", "BSIC"),
                                OntologyNode(
                                    "Physical Cell Identifier", "PCI, LTE Only"
                                ),
                                OntologyNode(
                                    "Reference Signal Received Power", "RSRP, LTE Only"
                                ),
                                OntologyNode(
                                    "Reference Signal Received Quality",
                                    "RSRQ, LTE Only",
                                ),
                                OntologyNode(
                                    "Reference Signal Signal to Noise Ratio",
                                    "RSSNR, LTE Only",
                                ),
                                OntologyNode("Channel Quality Indicator", "CQI"),
                                OntologyNode("Timing Advance", "TA - high is far"),
                            ],
                        ),
                        OntologyNode(
                            "WiFi Network",
                            None,
                            [
                                OntologyNode("SSID"),
                                OntologyNode("WiFi Channel"),
                                OntologyNode("WiFi Encryption"),
                                OntologyNode("Modulation and Coding Scheme", "MCS"),
                                OntologyNode("BSSID"),
                                OntologyNode("Wifi Frequency"),
                            ],
                        ),
                        OntologyNode(
                            "Quality of Service",
                            None,
                            [
                                OntologyNode("Upload Throughput"),
                                OntologyNode("Download Throughput"),
                                OntologyNode("Latency Average"),
                                OntologyNode("Link Speed"),
                                OntologyNode("Signal Strength"),
                                OntologyNode("Jitter", "Variance of latency"),
                                OntologyNode("Packet Loss", "Lost Percentage"),
                            ],
                        ),
                    ],
                ),
            ],
        ),
    ],
)


In [20]:
wireless_ontology = dtl.ontology.create(wireless_ontology)

![Ontology view](images/ontology.png)

Partial preview of the ontology create above in the Datalogue interface.

<a id='Training_Data'></a>
## Attach training data

Attaching training data from telemetry source

In [21]:
## get ontology leaves

leaves = wireless_ontology.leaves()

# for idx, leaf in enumerate(leaves):
#    print(idx, "|", leaf.node_id, "|", leaf.name, "|")

In [22]:
# create leaves dictionary

leaf_node_dict = {}
for leaf in leaves:
    leaf_node_dict[leaf.name] = leaf

In [23]:
# Identifying data stores for reference

datastore_list = dtl.datastore.list() # this would be better as a dictionary so we can use it programatically
    
# for i in range(len(datastore_list)):
#     print("➜ ", datastore_list[i].id, "|", datastore_list[i].name)
# print('\n Currently total of ', len(datastore_list), ' datastores available.')

In [24]:
def GetDatastoreID (datastoreName):
    """
    Retrieves store ID by reference to store name. If there are multiple identically named stores, returns ID of first one. Note: it is bad practice to have multiple identical data stores
    
    :param datastoreName: Name of datastore to be added
    """
    return list(filter(lambda d: d.name == datastoreName, datastore_list))[0].id 

In [25]:
def CreateLeafNodeDict(ontology):
    """
    Takes ontology and creates a leaf node dictionary for use in attaching training data
    
    :param ontology: Ontology to be used below
    """
    
    # get ontology leaves
    
    leaves = wireless_ontology.leaves()
    
    # create leaves dictionary
    
    leaf_node_dict = {}
    for leaf in leaves:
        leaf_node_dict[leaf.name] = leaf

In [26]:
CreateLeafNodeDict(wireless_ontology) # Note, probably best to shove into AttachTrainingDataDictionary

In [27]:
def AttachTrainingDataDictionary(col_dictionary, store_path, store_name, ontology_dict = leaf_node_dict):
    """
    This takes a mapping of the columns in a data store to an ontology, the details of a data store and the details of an ontology and attaches the data to the ontology as training data.

    Requires function GetStoreId()
    Requires col_dictionary
    Requires leaf_node_dict created with function CreateLeafNodeDict()
    
    :param col_dictionary: a dictionary mapping the ontology to the data store, structured {"Ontology Leaf Node Name": "column_Name"}
    :param store_path: the path to the datastore
    :param store_name: name of data store
    :param ontology_dict: dictionary which has the leaf nodes of the relevant ontology, defined with CreateLeafNodeDict(ontology)
    """
    for leaf in col_dictionary:
        col = col_dictionary[leaf]
        path = ([store_path, col])
        node = ontology_dict[leaf]
        ref = DataRef(node, [path])
        stream_id_for_data_transfer = dtl.training.data.add(
            GetDatastoreID(store_name), store_name, [ref]
        )

In [28]:
# details about data store

us_telemetry_data_store_name = "US Telemetry 5k"
us_telemetry_data_store_id = GetDatastoreID(us_telemetry_data_store_name)
us_telemetry_data_path = "5K_us_telemetry.csv"

# ontology nodes, and training data columns

us_telemetry_training_col_dict = {
    "Packet Loss": "main_QOS_PacketLoss_LostPercentage",
    "Jitter": "main_QOS_Jitter_Average",
    "Signal Strength": "main_QOS_SignalStrength",
    "Link Speed": "main_QOS_LinkSpeed",
    "Latency Average": "main_QOS_Latency_Average",
    "Download Throughput": "main_QOS_DownloadThroughput",
    "Upload Throughput": "main_QOS_UploadThroughput",
    "Wifi Frequency": "main_WifiFrequency",
    "BSSID": "main_BSSID",
    "Timing Advance": "main_QOS_TA",
    "Channel Quality Indicator": "main_QOS_CQI",
    "Reference Signal Signal to Noise Ratio": "main_QOS_RSSNR",
    "Reference Signal Received Quality": "main_QOS_RSRQ",
    "Reference Signal Received Power": "main_QOS_RSRP",
    "Physical Cell Identifier": "main_PCI",
    "Base Station Identity Code": "main_BSIC",
    "Mobile Country Code": "main_MCC",
    "Mobile Channel": "main_MobileChannel",
    "Mobile Connection Generation": "conn_Generation_Category",
    "Session Delta Received Bytes": "main_QOS_DeltaReceivedBytes",
    "Session Delta Transmitted Bytes": "main_QOS_DeltaTransmittedBytes",
    "QoS Timestamp": "main_QOS_QOSDate",
    "Session Timezone": "main_Timezone",
    "Session End": "main_ConnectionEnd",
    "Session Start": "main_ConnectionStart",
    "Session Connection Technology": "main_ConnectionTechnology",
    "Session Connection Type": "main_ConnectionType",
    "Session Country": "main_Country",
    "Device Velocity Bearing": "main_QOS_Velocity_Bearing",
    "Device Velocity Speed": "main_QOS_Velocity_Speed",
    "Device Vertical Accuracy": "main_QOS_Location_VerticalAccuracy",
    "Device Horizontal Accuracy": "main_QOS_Location_HorizontalAccuracy",
    "Device Battery State": "main_QOS_DeviceBatteryState",
    "Device Battery Usage": "main_QOS_DeviceBatteryLevel",
    "Device CPU Usage": "main_QOS_DeviceCPU",
    "Device Unused Memory": "main_QOS_DeviceFreeMemory",
    "Device Used Memory": "main_QOS_DeviceUsedMemory",
    "Device Unused Storage": "main_QOS_DeviceFreeStorage",
    "Device Used Storage": "main_QOS_DeviceUsedStorage",
    "Device System Uptime": "main_QOS_SystemUptime",
    "Device Language": "main_Device_DeviceLanguage",
    "Device Storage": "main_Device_Storage",
    "Device Memory": "main_Device_Memory",
    "Screen Resolution Height": "main_Device_ScreenResolution_Height",
    "Screen Resolution Width": "main_Device_ScreenResolution_Width",
    "Device Model": "main_Device_Model",
    "OS": "main_Device_OS",
    "Manufacturer": "main_Device_Manufacturer",
    "Network Provider": "sp_ServiceProviderBrandName",
    "Network Provider": "main_ServiceProvider",
    "Mobile Network Code": "main_MNC",
    "Geo Hash": "main_Geohash",
    "Altitude": "main_QOS_Location_Altitude",
    "Longitude": "main_QOS_Location_Longitude",
    "Latitude": "main_QOS_Location_Latitude",
    "Subscriber City": "main_City",
}

# attaches above as training data

# for leaf in us_telemetry_training_col_dict:
#     col = us_telemetry_training_col_dict[leaf]
#     path = ([us_telemetry_data_path, col])
#     node = leaf_node_dict[leaf]
#     ref = DataRef(node, [path])
#     us_telemetry_stream_id_for_data_transfer = dtl.training.data.add(GetDatastoreID(us_telemetry_data_store_name), us_telemetry_data_store_name, [ref])

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

In [29]:
us_telemetry_training_col_dict2 = {
    "Packet Loss": "main_QOS_PacketLoss_LostPercentage",
    "Jitter": "main_QOS_Jitter_Average",
    "Signal Strength": "main_QOS_SignalStrength",
    "Link Speed": "main_QOS_LinkSpeed",
    "Latency Average": "main_QOS_Latency_Average",
    "Download Throughput": "main_QOS_DownloadThroughput",
    "Upload Throughput": "main_QOS_UploadThroughput",
    "Wifi Frequency": "main_WifiFrequency",
    "BSSID": "main_BSSID",
    "Timing Advance": "main_QOS_TA",
}

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict2,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

In [30]:
us_telemetry_training_col_dict3 = {
    "Channel Quality Indicator": "main_QOS_CQI",
    "Reference Signal Signal to Noise Ratio": "main_QOS_RSSNR",
    "Reference Signal Received Quality": "main_QOS_RSRQ",
    "Reference Signal Received Power": "main_QOS_RSRP",
    "Physical Cell Identifier": "main_PCI",
    "Base Station Identity Code": "main_BSIC",
    "Mobile Country Code": "main_MCC",
    "Mobile Channel": "main_MobileChannel",
    "Mobile Connection Generation": "conn_Generation_Category",
    "Session Delta Received Bytes": "main_QOS_DeltaReceivedBytes",
}

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict3,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

In [31]:
us_telemetry_training_col_dict4 = {
    "Session Delta Transmitted Bytes": "main_QOS_DeltaTransmittedBytes",
    "QoS Timestamp": "main_QOS_QOSDate",
    "Session Timezone": "main_Timezone",
    "Session End": "main_ConnectionEnd",
    "Session Start": "main_ConnectionStart",
    "Session Connection Technology": "main_ConnectionTechnology",
    "Session Connection Type": "main_ConnectionType",
    "Session Country": "main_Country",
    "Device Velocity Bearing": "main_QOS_Velocity_Bearing",
    "Device Velocity Speed": "main_QOS_Velocity_Speed",
}

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict4,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

In [32]:
us_telemetry_training_col_dict5 = {
    "Device Vertical Accuracy": "main_QOS_Location_VerticalAccuracy",
    "Device Horizontal Accuracy": "main_QOS_Location_HorizontalAccuracy",
    "Device Battery State": "main_QOS_DeviceBatteryState",
    "Device Battery Usage": "main_QOS_DeviceBatteryLevel",
    "Device CPU Usage": "main_QOS_DeviceCPU",
    "Device Unused Memory": "main_QOS_DeviceFreeMemory",
    "Device Used Memory": "main_QOS_DeviceUsedMemory",
    "Device Unused Storage": "main_QOS_DeviceFreeStorage",
    "Device Used Storage": "main_QOS_DeviceUsedStorage",
    "Device System Uptime": "main_QOS_SystemUptime",
}

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict5,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

In [33]:
us_telemetry_training_col_dict6 = {
    "Device Language": "main_Device_DeviceLanguage",
    "Device Storage": "main_Device_Storage",
    "Device Memory": "main_Device_Memory",
    "Screen Resolution Height": "main_Device_ScreenResolution_Height",
    "Screen Resolution Width": "main_Device_ScreenResolution_Width",
    "Device Model": "main_Device_Model",
    "OS": "main_Device_OS",
    "Manufacturer": "main_Device_Manufacturer",
    "Network Provider": "sp_ServiceProviderBrandName",
}

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict6,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

In [34]:
us_telemetry_training_col_dict7 = {
    "Network Provider": "main_ServiceProvider",
    "Mobile Network Code": "main_MNC",
    "Geo Hash": "main_Geohash",
    "Altitude": "main_QOS_Location_Altitude",
    "Longitude": "main_QOS_Location_Longitude",
    "Latitude": "main_QOS_Location_Latitude",
    "Subscriber City": "main_City",
}

AttachTrainingDataDictionary(
    us_telemetry_training_col_dict7,
    us_telemetry_data_path,
    us_telemetry_data_store_name,
)

## Enriching training data with other sources

In [35]:
# Canadian Names datastore

canadian_names_data_store_name = "Canadian Names"
canadian_names_data_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500canadians.csv"

canadian_names_col_dict = {
'First Name': 'first_name',
'Family Name': 'last_name',
'Subscriber Street': 'address',
'Subscriber Company': 'company_name',
'Subscriber City': 'city',
'Session State or Territory': 'province',
'Subscriber Zip': 'postal',
'Subscriber Phone Number': 'phone1',
'Subscriber Phone Number': 'phone2',
'Subscriber Email': 'email',
}

AttachTrainingDataDictionary(
    canadian_names_col_dict,
    canadian_names_data_path,
    canadian_names_data_store_name,
)


In [36]:
# British Names datastore

british_names_data_store_name = "British Names"
british_names_data_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500brits.csv"

british_names_col_dict = {
'First Name': 'first_name',
'Family Name': 'last_name',
'Subscriber Company': 'company_name',
'Subscriber Street': 'address',
'Subscriber City': 'city',
'Session State or Territory': 'county',
'Subscriber Zip': 'postal',
'Subscriber Phone Number': 'phone1',
'Subscriber Phone Number': 'phone2',
'Subscriber Email': 'email',
}

AttachTrainingDataDictionary(
    british_names_col_dict,
    british_names_data_path,
    british_names_data_store_name,
)

In [37]:
# Australian Names datastore

australian_names_data_store_name = "Australian Names"
australian_names_data_store_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500australians.csv"

australian_names_col_dict = {
'First Name': 'first_name',
'Family Name': 'last_name',
'Subscriber Company': 'company_name',
'Subscriber Street': 'address',
'Subscriber City': 'city',
'Session State or Territory': 'state',
'Subscriber Zip': 'post',
'Subscriber Phone Number': 'phone1',
'Subscriber Phone Number': 'phone2',
'Subscriber Email': 'email',
}

AttachTrainingDataDictionary(
    australian_names_col_dict,
    australian_names_data_store_path,
    australian_names_data_store_name,
)

In [38]:
# US Names datastore

us_names_data_store_name = "US Names"
us_names_data_store_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/500americans.csv"

us_names_col_dict = {
'First Name': 'first_name',
'Family Name': 'last_name',
'Subscriber Company': 'company_name',
'Subscriber Street': 'address',
'Subscriber City': 'city',
'Session State or Territory': 'county',
'Session State or Territory': 'state',
'Subscriber Zip': 'zip',
'Subscriber Phone Number': 'phone1',
'Subscriber Phone Number': 'phone2',
'Subscriber Email': 'email',
}

AttachTrainingDataDictionary(
    us_names_col_dict,
    us_names_data_store_path,
    us_names_data_store_name,
)

In [39]:
# Users datastore

users_data_store_name = "Users"
users_data_store_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/pii/users.csv"

users_col_dict = {
'First Name': "first_name",
'Family Name': "last_name",
'Subscriber Company': 'company_name',
'Subscriber Street': 'address',
'Subscriber City': 'city',
'Session State or Territory': 'county',
'Session State or Territory': 'state',
'Subscriber Zip': 'zip',
'Subscriber Phone Number': 'phone1',
'Subscriber Phone Number': 'phone',
'Subscriber Email': 'email',
}

AttachTrainingDataDictionary(
    users_col_dict,
    users_data_store_path,
    users_data_store_name,
)

In [40]:
# Countries datastore

countries_data_store_name = "Countries"
countries_data_store_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/countries.csv"

countries_col_dict = {
'Session Country': 'Name',
'Session Country': 'Code'
}

AttachTrainingDataDictionary(
    countries_col_dict,
    countries_data_store_path,
    countries_data_store_name,
)

In [41]:
# Some Cities datastore

some_cities_data_store_name = "Some Cities"
some_cities_data_store_path = "https://raw.githubusercontent.com/datalogue/demo-data/master/people_places/cities.csv"

some_cities_col_dict = {
'Subscriber City': 'City',
'Session State or Territory': 'State'
}

AttachTrainingDataDictionary(
    some_cities_col_dict,
    some_cities_data_store_path,
    some_cities_data_store_name,
)

![Ontology view detail: training data](images/ontology2.png)

Partial preview of the ontology created above, highlighting attached training data, in the Datalogue interface.

<a id='Training'></a>
# Train a model

Warning, this is GPU intensive

In [ ]:
# # Use Datalogue Semantic Engine to train a classification model on the Telemetry Ontology 

# dtl.training.run(
#   ontology_id = wireless_ontology.ontology_id # will change to wireless_ontology.id in next version
# )

![Training data view](images/metrics.png)

Model metrics view showing above training in progress.

## Deploy model

Manual step currently.

In [44]:
# retrieve training

list_of_trainings = dtl.training.get_trainings(
  ontology_id = wireless_ontology.ontology_id, # will change to wireless_ontology.id in next version
)

for training in list_of_trainings:
    print("Training for ontology id: " + str(training.ontology_id))
    print("   " + training.status)
    print("   " + training.start_time)
    print("   " + training.model_type)



# deploy model
## Note: this is in version 0.9 of the General Release, for now, use GUI

# dtl.training.deploy(, wireless_ontology.id)

Training for ontology id: 77acad0f-53f4-4bfe-9ed3-7811017529d2
   Ended
   2019-05-29T19:58:22.944Z
   cbc


![Model Metrics](images/metrics2.png)

Model metrics view showing performance of above model.

<a id='Pipelines'></a>
# Pipelining

## Attach large datastores to be inferred upon

In [60]:
# Attach a datastore: European Sample Dataset

gcsdef_2 = GCSDatastoreDef(
    bucket = "dtl-handset-telemetry",
    file_name = "5K_eu_telemetry.csv",
    file_format = FileFormat.Csv,
)

EU_sample_handset_data = dtl.datastore.create(
    Datastore(
        name = "European Handset Telemetry Sample", 
        definition = gcsdef_2, 
        credential_id = my_gcs_credentials.id
    )
)

In [61]:
# Attach a datastore: European Fullscale Dataset

gcsdef_3 = GCSDatastoreDef(
    bucket = "dtl-handset-telemetry",
    file_name = "gtc_europe.csv",
    file_format = FileFormat.Csv,
)

EU_fullscale_handset_data = dtl.datastore.create(
    Datastore(
        name = "European Handset Telemetry", 
        definition = gcsdef_3, 
        credential_id = my_gcs_credentials.id
    )
)

In [62]:
# Attach a datastore: US Fullscale Dataset

gcsdef_4 = GCSDatastoreDef(
    bucket = "dtl-handset-telemetry",
    file_name = "us_340M.csv",
    file_format = FileFormat.Csv,
)

US_fullscale_handset_data = dtl.datastore.create(
    Datastore(
        name = "US Handset Telemetry", 
        definition = gcsdef_4, 
        credential_id = my_gcs_credentials.id
    )
)

In [63]:
# Collect into datastore collection

dtl.datastore_collection.create(
    DatastoreCollection(
        name = "Handset telemetry data for inference",
        description = "Handset training data for use in model creation",
        storeIds = [EU_sample_handset_data.id, EU_fullscale_handset_data.id, US_fullscale_handset_data.id],
    )
)

BOT.server_summary()

no pipelines available

Datalogue Server Summary :: Stores: 13, Collections: 4, Streams: 0



In [64]:
# Checking that the collection was created

for dataset in dtl.datastore_collection.list():
    print('\033[1m' "Collection Name: ", dataset.name) 
    print('\033[0m' "             ID: ", dataset.id)
    print("       Contains: ", )
    for datastore in dataset.storeIds:
        print("                  ➜ " + datastore["name"])
        print("                    ↪ID: ", datastore["id"])
    print()


Collection Name:  Handset telemetry training data
             ID:  f7f9b5e4-b06f-4133-9d83-b943d18a953b
       Contains: 
                  ➜ US Telemetry 5k
                    ↪ID:  9e5aa44e-b321-44d6-b863-1577bcad7572

Collection Name:  Supplemental training data (mainly customer data)
             ID:  21e41c0e-304a-479c-8cb9-7005d25b946c
       Contains: 
                  ➜ US Names
                    ↪ID:  0000effb-eeda-43fd-8545-994ae1686335
                  ➜ Australian Names
                    ↪ID:  30a49e46-765c-4e30-9917-6476948e649f
                  ➜ Canadian Names
                    ↪ID:  3511a4b6-a498-4228-a765-878f2b710c80
                  ➜ Countries
                    ↪ID:  5212d5f9-91b7-4e1e-aa10-09a2e49a3bba
                  ➜ British Names
                    ↪ID:  b4770dfd-1078-4eaa-a8a8-09ec1faf1dd0
                  ➜ Users
                    ↪ID:  c7d8384f-2ef9-40d3-8850-c7d983cf7362
                  ➜ Some Cities
                    ↪ID:  fe346009-

![Datastore Colletion view](images/Datastore_collection3.png)

Preview of the Datastore collection above in the Datalogue interface.

# Analyze datastore in GUI before pipelining (see comment below)

Analyse datastore
This allows us to classify only the first chunk of a dataset, to spare GPU resources if a schema is static. do this in the GUI by going into the data store details through the settings menu (three dots on the top right) then select the dataset, and hit "analyse" in the three dots menu from that datastore.

![Analyze view](images/analyze.png)

Analysis showing inferred classes on European and US datasets.


# Pipeline definition

In [53]:
# Define source
source1 = EU_sample_handset_data

# Define destination

gcsdef_5 = GCSDatastoreDef(
    bucket="dtl-handset-telemetry",
    file_name="cleaned_handset_telemetry_data.csv",
    file_format=FileFormat.Csv,
)

target1 = dtl.datastore.create(
    Datastore(
        name="Cleaned Handset Telemetry data",
        definition=gcsdef_5,
        credential_id=my_gcs_credentials.id,
    )
)


In [65]:
# create list of class nodes for use in structure transformation
class_nodes_list = []

# populate list with leaves

for leaf in leaves:
    class_nodes_list.append(
        ClassNodeDescription(
            path=[leaf.name],
            tag=leaf.name,
            pick_strategy=PickStrategy.HighScore,
            data_type=DataType.String,
        )
    )

In [ ]:
# Define pipeline
definition1 = Definition(
    transformations=[
#        Classify(None, True), # If you have the analyze above, you don't need to classify. It is much more efficient to do this.
        Structure(
            class_nodes_list
        ),
    ],
    pipelines=[],
    target=target1,
)

# Define stream
my_stream = Stream(source1, [definition1])

# Push

stream_collection = dtl.stream_collection.create(
    [my_stream], "Telemetry classification pipeline"
)


![Pipeline view](images/pipeline.png)

Pipeline view showing classification and structure transformations.

In [79]:
# Define source
source2 = EU_fullscale_handset_data

# Define destination

gcsdef_5 = GCSDatastoreDef(
    bucket="dtl-handset-telemetry",
    file_name="cleaned_EU_handset_telemetry_data.csv",
    file_format=FileFormat.Csv,
)

target1 = dtl.datastore.create(
    Datastore(
        name="Cleaned EU Handset Telemetry data",
        definition=gcsdef_5,
        credential_id=my_gcs_credentials.id,
    )
)

# Define pipeline
definition1 = Definition(
    transformations=[
#        Classify(None, True), # If you have the analyze above, you don't need to classify. It is much more efficient to do this.
        Structure(
            class_nodes_list
        ),
    ],
    pipelines=[],
    target=target1,
)

# Define stream
my_stream = Stream(source2, [definition1])

# Push

stream_collection = dtl.stream_collection.create(
    [my_stream], "EU Telemetry classification pipeline"
)



In [80]:
# Define source
source3 = US_fullscale_handset_data

# Define destination

gcsdef_5 = GCSDatastoreDef(
    bucket="dtl-handset-telemetry",
    file_name="cleaned_US_handset_telemetry_data.csv",
    file_format=FileFormat.Csv,
)

target1 = dtl.datastore.create(
    Datastore(
        name="Cleaned US Handset Telemetry data",
        definition=gcsdef_5,
        credential_id=my_gcs_credentials.id,
    )
)

# Define pipeline
definition1 = Definition(
    transformations=[
#        Classify(None, True), # If you have the analyze above, you don't need to classify. It is much more efficient to do this.
        Structure(
            class_nodes_list
        ),
    ],
    pipelines=[],
    target=target1,
)

# Define stream
my_stream = Stream(source3, [definition1])

# Push

stream_collection = dtl.stream_collection.create(
    [my_stream], "US Telemetry classification pipeline"
)



In [107]:
stream_collection._as_payload()

{'id': '0a275942-a069-48bd-9f70-07168334a149',
 'name': 'US Telemetry classification pipeline',
 'streams': [{'id': '35214455-ef86-48c6-8943-c1991ce34933',
   'isReady': True,
   'stream': {'source': {'type': 'GCS',
     'id': '4d034c34-4976-423e-b93a-379ec8645873',
     'bucket': 'dtl-handset-telemetry',
     'fileName': 'us_340M.csv',
     'format': 'Csv',
     'params': {}},
    'pipelines': [{'transformations': [{'type': 'Structure',
        'outputSchema': [{'type': 'Class',
          'outputDataType': 'String',
          'outputPath': ['Full Name'],
          'class': 'Full Name',
          'pickStrategy': 'HighScore'},
         {'type': 'Class',
          'outputDataType': 'String',
          'outputPath': ['First Name'],
          'class': 'First Name',
          'pickStrategy': 'HighScore'},
         {'type': 'Class',
          'outputDataType': 'String',
          'outputPath': ['Family Name'],
          'class': 'Family Name',
          'pickStrategy': 'HighScore'},
        

In [128]:
dtl.jobs.list()

[Job(job_id: UUID('10da9efb-9195-4603-82a0-15ae69283cb9'), stream_collection_id: UUID('b0d1f780-65f6-4adf-a2a6-7321e2799930'), status: Succeeded, ,
 Job(job_id: UUID('1e7cd867-3526-4276-9bd7-988a94512989'), stream_collection_id: UUID('6f79674e-cbe8-44de-8c1e-10df6ed3d267'), status: Running, ,
 Job(job_id: UUID('6075ac62-e86c-49c8-9bce-91df7710a1ce'), stream_collection_id: UUID('67e7d08e-da06-415f-8e30-7dd9ad94786e'), status: Running, ,
 Job(job_id: UUID('8ed1ca20-dd44-4ec1-a032-a795147d5093'), stream_collection_id: UUID('67e7d08e-da06-415f-8e30-7dd9ad94786e'), status: Failed, ,
 Job(job_id: UUID('cc79785d-61d1-4350-b7c4-dcda8d1ebd1c'), stream_collection_id: UUID('67e7d08e-da06-415f-8e30-7dd9ad94786e'), status: Succeeded, ,
 Job(job_id: UUID('3c11d90c-8c74-4540-b59a-8d9d84bbc568'), stream_collection_id: UUID('0a275942-a069-48bd-9f70-07168334a149'), status: Running, ,
 Job(job_id: UUID('4e24e0e8-847a-4abe-8929-67c9808b603a'), stream_collection_id: UUID('a45e73b2-6455-410e-b207-b43aab4f27

In [134]:
print(dtl.jobs.list()[3].percentage_progress)

0
